### Creating classes

In [1]:
from collections import defaultdict
from math import sqrt
import numpy as np
from scipy.stats import norm

In [2]:
class Derivative:
    """ Enabling the usage of +, *, -, etc. """
    def __add__(self, other):
        return Add(self, other)
    
    def __radd__(self, other):
        return rAdd(self, other)
    
    def __sub__(self, other):
        return Sub(self, other)

    def __rsub__(self, other):
        return rSub(self, other)
    
    def __mul__(self, other):
        return Mul(self, other)
    
    def __rmul__(self, other):
        return rMul(self, other)
    
    def __truediv__(self, other):
        return Div(self, other)

    def __rtruediv__(self, other):
        return rDiv(self, other)
   
    def Sin(self):
        return Sin(self)
    
    def __pow__(self, power):
        return Pow(self, power)
    
    def Exp(self):
        return Exp(self)
    
    def Cdf(self):
        return Cdf(self)
    
    def __ge__(self, other):
        return Ge(self, other)
    
    def __le__(self, other):
        return Le(self, other)
    
    def __hash__(self):                 # we need this for creating the dictionary with keys as Derivatives object
        return hash(str(self))

Leaf nodes

In [3]:
class Var(Derivative):
    """ A leaf node (a node which doesn't have any child) """
    
    def __init__(self, value):
        self.value = value      # the scalar value of the node

Adding the nodes

In [4]:
class Add(Derivative):
    """ The node that results from adding two nodes """
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float, np.ndarray)): 
            self.value = node_a + node_b.value
            self.grad = [(node_b, 1)]
        elif isinstance(node_b, (int, float, np.ndarray)): 
            self.value = node_a.value + node_b
            self.grad = [(node_a, 1)]
        else:      
            self.value = node_a.value + node_b.value    # value of the node
            self.grad = [(node_a, 1), (node_b, 1)]      # partial derivatives of nodes - value 1 for derivative in respect to node_a and 1 for node_b

In [5]:
class rAdd(Derivative):
    """ The node that results from adding two nodes """
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float)): 
            self.value = node_a + node_b.value
            self.grad = [(node_b, 1)]
        elif isinstance(node_b, (int, float)): 
            self.value = node_a.value + node_b
            self.grad = [(node_a, 1)]
        else:      
            self.value = node_a.value + node_b.value    # value of the node
            self.grad = [(node_a, 1), (node_b, 1)]      # partial derivatives of nodes - value 1 for derivative in respect to node_a and 1 for node_b

Substract the nodes

In [6]:
class Sub(Derivative):
    """ The node that results from subtracting two nodes """
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float)): 
            self.value = node_a - node_b.value
            self.grad = [(node_b, -1)]
        elif isinstance(node_b, (int, float)): 
            self.value = node_a.value - node_b
            self.grad = [(node_a, 1)]
        else:      
            self.value = node_a.value - node_b.value    # value of the node
            self.grad = [(node_a, 1), (node_b, -1)]     # partial derivatives of nodes - should have again the structure as [(node_a, value), (node_b, value)]

In [7]:
class rSub(Derivative):
    """ The node that results from subtracting two nodes """
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float)): 
            self.value = node_a - node_b.value
            self.grad = [(node_b, -1)]
        elif isinstance(node_b, (int, float)): 
            self.value = node_a.value - node_b
            self.grad = [(node_a, 1)]
        else:      
            self.value = node_a.value - node_b.value    # value of the node
            self.grad = [(node_a, 1), (node_b, -1)]     # partial derivatives of nodes - should have again the structure as [(node_a, value), (node_b, value)]

Multiplication of nodes

In [8]:
class Mul(Derivative):
    """ The node that results from multiplying two nodes """
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float, np.ndarray)): 
            self.value = node_a * node_b.value
            self.grad = [(node_b, node_a)]
        elif isinstance(node_b, (int, float, np.ndarray)): 
            self.value = node_a.value * node_b
            self.grad = [(node_a, node_b)]
        else:      
            self.value = node_a.value * node_b.value
            self.grad = [(node_a, node_b.value), (node_b, node_a.value)]        # f = x*y   df / dx = y   df / dy = x

In [9]:
class rMul(Derivative):
    """ The node that results from multiplying two nodes """
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float, np.ndarray)): 
            self.value = node_a * node_b.value
            self.grad = [(node_b, node_a)]
        elif isinstance(node_b, (int, float, np.ndarray)): 
            self.value = node_a.value * node_b
            self.grad = [(node_a, node_b)]
        else:      
            self.value = node_a.value * node_b.value
            self.grad = [(node_a, node_b.value), (node_b, node_a.value)]                                # f = x*y   df / dx = y   df / dy = x

Dividing nodes

In [10]:
class Div(Derivative):
    """ The node that results from dividing one node by another """
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float)): 
            self.value = node_a / node_b.value
            self.grad = [(node_b, -node_a/(node_b.value**2))]
        elif isinstance(node_b, (int, float)): 
            self.value = node_a.value / node_b
            self.grad = [(node_a, 1/node_b)]
        else:      
            self.value = node_a.value / node_b.value
            self.grad = [(node_a, 1/node_b.value), (node_b, -node_a.value/(node_b.value**2))]           # f = x/y   df / dx = 1/y    df / dy = -x / (y^2)

In [11]:
class rDiv(Derivative):
    """ The node that results from dividing one node by another """
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float)): 
            self.value = node_a / node_b.value
            self.grad = [(node_b, -node_a/(node_b.value**2))]
        elif isinstance(node_b, (int, float)): 
            self.value = node_a.value / node_b
            self.grad = [(node_a, 1/node_b)]
        else:      
            self.value = node_a.value / node_b.value
            self.grad = [(node_a, 1/node_b.value), (node_b, -node_a.value/(node_b.value**2))]           # f = x/y   df / dx = 1/y    df / dy = -x / (y^2)

Logarithm of one mode

In [12]:
class Log(Derivative):
    """ The node that results from sin(node) """
    
    def __init__(self, node):
        if isinstance(node, (int, float)): 
            self.value = np.log(node) 
            self.grad = [(node, 0)]
        else:
            self.value = np.log(node.value)                          
            self.grad = [(node, 1/node.value)]                 

Sinus of one node

In [13]:
class Sin(Derivative):
    """ The node that results from sin(node) """
    
    def __init__(self, node):
        if isinstance(node, (int, float)): 
            self.value = np.sin(node) 
            self.grad = [(node, 0)]
        else:
            self.value = np.sin(node.value)                          # use np.sin() function
            self.grad = [(node, np.cos(node.value))]                 # only one derivative, since it takes only one node - use np.cos() function

Cosinus of one node

In [14]:
class Cos(Derivative):
    """ The node that results from cos(node) """
    
    def __init__(self, node):
        if isinstance(node, (int, float)): 
            self.value = np.cos(node)
            self.grad = [(node, 0)]
        else:
            self.value = np.cos(node.value)                          
            self.grad = [(node, -np.sin(node.value))]                

Powers

In [15]:
class Pow(Derivative):
    """ The node that results as node ^ power """
    
    def __init__(self, node, power):
        self.value = node.value**power                                         # reminder of power operation in python: **
        self.grad = [(node, power*node.value**(power-1))]                      # (one derivative)    f = x^n   df/dx = n * x ^ (n-1)

Exponential

In [16]:
class Exp(Derivative):
    """ The node that results from exp(node) """

    def __init__(self, node):
        if isinstance(node, (int, float)): 
            self.value = np.exp(node)  
            self.grad = [(node, 0)]
        else:
            self.value = np.exp(node.value)                         
            self.grad = [(node, np.exp(node.value))]   

In [17]:
class ExpK(Derivative):
    """ The node that results from exp(k*node) """
    
    def __init__(self, node, k):
        self.value = np.exp(k*node)                          
        self.grad = [(node, k*np.exp(k*node))]                

Squareroot

In [18]:
class Sqrt(Derivative):
    """ The node that results from sqrt(node) """
    
    def __init__(self, node):
        if isinstance(node, (int, float)): 
            self.value = np.sqrt(node)
            self.grad = [(node, 0)]
        else:
            self.value = np.sqrt(node.value)                  
            self.grad = [(node, 1/(2*np.sqrt(node.value)))]  

$ a^x $

In [24]:
class Pot(Derivative):
    """ The node that results from a ^ node """
    
    def __init__(self, a, node):
        self.value = a**node.value                          
        self.grad = [(node, a**node.value * np.log(a))]                

Cdf

In [25]:
class Cdf(Derivative):
    """ The node that results from cdf(node) """

    def __init__(self, node):
        if isinstance(node, (int, float)): 
            self.value = norm.cdf(node) 
            self.grad = [(node, norm.pdf(node) )]
        else:
            self.value = norm.cdf(node.value)            
            self.grad = [(node, norm.pdf(node.value))]

Overskrivning af >= og <=

In [26]:
class Ge(Derivative):
    """ >= """
            
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float)): 
            self.value = node_a >= node_b.value
        elif isinstance(node_b, (int, float)): 
            self.value = node_a.value >= node_b
        else:      
            self.value = node_a.value >= node_b.value

In [27]:
class Le(Derivative):
    """ <= """

    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float)): 
            self.value = node_a <= node_b.value
        elif isinstance(node_b, (int, float)): 
            self.value = node_a.value <= node_b
        else:      
            self.value = node_a.value <= node_b.value

### Create a function for getting the gradients values

In [28]:
def Get_Gradient(parent_node):
    """ Go down the graph, and compute derivative of `parent_node` with respect to each node """
    
    # we will create a dictionary 'gradient' which will have the nodes as keys and its derivatives as values
    gradients = defaultdict(lambda: 0)    # initialize the dictionary so when calling a non-existing key the value of 0 is assigned
    
    # stack will represent the list of tuples (node, node_derivative) 
    stack = parent_node.grad.copy()     
    
    while stack:                             # loop for each different branch
        # get node and node_derivative from the top of the stack - function pop()
        temp = stack.pop()                   
        node = temp[0]
        node_derivative = temp[1]            
        # add to the value of derivative of the node (gradients[node]) value node_derivative
        gradients[node] = gradients[node] + node_derivative  
        
        if not isinstance(node, Var):        # if the node has children, put them onto the stack
            # loop for each node in one branch
            for child_node, child_node_derivative in node.grad:                   
                # append child_node and child_node_derivative * node_derivative to the stack
                stack.append((child_node, child_node_derivative * node_derivative))
                
    return dict(gradients)

### Test: Black-Scholes

In [29]:
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib import style 
style.use('ggplot')
from scipy.stats import norm
import scipy

In [30]:
def euro_call(S0, K, T, r, sigma):
    d1 = (Log(S0/K)+(r+1/2*(sigma**2))*T)/(sigma*Sqrt(T))
    d2 = d1 - sigma*Sqrt(T)
    return S0 * Cdf(d1)  - K * Exp((-1)*r * T) * Cdf(d2)

In [31]:
S0_val = 100
K_val = 100
T_val = 1
r_val = 0.07
sigma_val = 0.2
n_simulations = 1000
n_steps = 252

In [32]:
S0 = Var(S0_val)
K = Var(K_val)
T = Var(T_val)
r = Var(r_val)
sigma = Var(sigma_val)

y = euro_call(S0, K, T, r, sigma)
gradients = Get_Gradient(y)

print('Value of f equals', y.value)
print('The partial derivative of y with respect to S0 =', gradients[S0])
print('The partial derivative of y with respect to K =', gradients[K])
print('The partial derivative of y with respect to T =', gradients[T])
print('The partial derivative of y with respect to r =', gradients[r])
print('The partial derivative of y with respect to sigma =', gradients[sigma])

Value of f equals 11.541470170672412
The partial derivative of y with respect to S0 = 0.6736447797120796
The partial derivative of y with respect to K = -0.5582300780053555
The partial derivative of y with respect to T = 7.512880170653974
The partial derivative of y with respect to r = 55.82300780053558
The partial derivative of y with respect to sigma = 36.05269624616482
